# Recommending Movies Using Affinity Analysis

Here, we will look at affinity analysis that determines when objects occur frequently together. This is colloquially called market basket analysis, after one of the use cases of determining when items are purchased together frequently.

The key concepts of this chapter are as follows:
- Affinity analysis
- Feature association mining using the Apriori algorithm
- Movie recommendations
- Sparse data formats

## Affinity analysis

Affinity analysis is the task of determining when objects are used in similar ways. In the previous section, we focused on whether the objects themselves are similar. The data for affinity analysis is often described in the form of a **transaction**. Intuitively, this comes from a transaction at a store—determining when objects are purchased together. 

However, it can be applied to many processes:
- Fraud detection
- Customer segmentation
- Software optimization
- Product recommendations

Affinity analysis is usually much more exploratory than classification. We often don't have the complete dataset we expect for many classification tasks. For instance, in movie recommendation, we have reviews from different people on different movies. However, it is unlikely we have each reviewer review all of the movies in our dataset. This leaves an important and difficult question in affinity analysis. If a reviewer hasn't reviewed a movie, is that an indication that they aren't interested in the movie (and therefore wouldn't recommend it) or simply that they haven't reviewed it yet?

## Algorithms for affinity analysis

With a naive rule creation, such as our previous algorithm, the growth in time needed to compute these rules increases exponentially. As we add more items, the time it takes to compute all rules increases significantly faster. Specifically, the total possible number of rules is 2n - 1. For our five-item dataset, there are 31 possible rules. For 10 items, it is 1023. For just 100 items, the number has 30 digits. Even the drastic increase in computing power couldn't possibly keep up with the increases in the number of items stored online. Therefore, we need algorithms that work smarter, as opposed to computers that work harder.

The classic algorithm for affinity analysis is called the **Apriori algorithm**. It addresses the exponential problem of creating sets of items that occur frequently within a database, called **frequent itemsets**. Once these frequent itemsets are discovered, creating association rules is straightforward.

The intuition behind Apriori is both simple and clever. First, we ensure that a rule has sufficient support within the dataset. Defining a minimum support level is the key parameter for Apriori. To build a frequent itemset, for an itemset (A, B) to have a support of at least 30, both A and B must occur at least 30 times in the database. This property extends to larger sets as well. For an itemset (A, B, C, D) to be considered
frequent, the set (A, B, C) must also be frequent (as must D).

These frequent itemsets can be built up and possible itemsets that are not frequent (of which there are many) will never be tested. This saves significant time in testing new rules.

Other example algorithms for affinity analysis include the **Eclat** and **FP-growth** algorithms. There are many improvements to these algorithms in the data mining literature that further improve the efficiency of the method.

## Choosing parameters

To perform association rule mining for affinity analysis, we first use the Apriori to generate frequent itemsets. Next, we create association rules (for example, if a person recommended movie X, they would also recommend movie Y) by testing combinations of premises and conclusions within those frequent itemsets.

For the first stage, the Apriori algorithm needs a value for the minimum support that an itemset needs to be considered frequent. Any itemsets with less support will not be considered. Setting this minimum support too low will cause Apriori to test a larger number of itemsets, slowing the algorithm down. Setting it too high will result in fewer itemsets being considered frequent.

In the second stage, after the frequent itemsets have been discovered, association
rules are tested based on their confidence. We could choose a minimum confidence
level, a number of rules to return, or simply return all of them and let the user decide
what to do with them.

In this chapter, we will return only rules above a given confidence level. Therefore,
we need to set our minimum confidence level. Setting this too low will result in rules
that have a high support, but are not very accurate. Setting this higher will result in
only more accurate rules being returned, but with fewer rules being discovered.

## The movie recommendation problem

Product recommendation is big business. Online stores use it to up-sell to
customers by recommending other products that they could buy. Making better
recommendations leads to better sales. When online shopping is selling to millions
of customers every year, there is a lot of potential money to be made by selling more
items to these customers.

Product recommendations have been researched for many years; however, the field
gained a significant boost when Netflix ran their Netflix Prize between 2007 and
2009. This competition aimed to determine if anyone can predict a user's rating of a
film better than Netflix was currently doing. The prize went to a team that was just
over 10 percent better than the current solution. While this may not seem like a large
improvement, such an improvement would net millions to Netflix in revenue from
better movie recommendations.

## Obtaining the dataset